In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from model.load_data import train_test_split, train_test_split_LSTM

from model.scoring_metrics import get_windiff, get_pk, get_k_kappa

batch_size = 64

# I optimize on this, I think?
LSTM_units = 20

features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

In [9]:
!pip install tensorflow-addons
!pip install typeguard

In [3]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

import warnings
from typing import Optional
from typeguard import typechecked
from tensorflow_addons.utils.types import Number

class WeightedKappaLoss(tf.keras.losses.Loss):
    @typechecked
    def __init__(
            self,
            num_classes: int,
            weightage: Optional[str] = "quadratic",
            name: Optional[str] = "cohen_kappa_loss",
            epsilon: Optional[Number] = 1e-6,
            dtype: Optional[tf.DType] = tf.float32,
            reduction: str = tf.keras.losses.Reduction.NONE,
    ):
        super().__init__(name=name, reduction=reduction)
        warnings.warn(
            "The data type for `WeightedKappaLoss` defaults to "
            "`tf.keras.backend.floatx()`."
            "The argument `dtype` will be removed in Addons `0.12`.",
            DeprecationWarning,
        )
        if weightage not in ("linear", "quadratic"):
            raise ValueError("Unknown kappa weighting type.")

        self.weightage = weightage
        self.num_classes = num_classes
        self.epsilon = epsilon or tf.keras.backend.epsilon()
        label_vec = tf.range(num_classes, dtype=tf.keras.backend.floatx())
        self.row_label_vec = tf.reshape(label_vec, [1, num_classes])
        self.col_label_vec = tf.reshape(label_vec, [num_classes, 1])
        col_mat = tf.tile(self.col_label_vec, [1, num_classes])
        row_mat = tf.tile(self.row_label_vec, [num_classes, 1])
        if weightage == "linear":
            self.weight_mat = tf.abs(col_mat - row_mat)
        else:
            self.weight_mat = (col_mat - row_mat) ** 2

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, dtype=self.col_label_vec.dtype)
        y_pred = tf.cast(y_pred, dtype=self.weight_mat.dtype)
        batch_size = tf.shape(y_true)[0]
        cat_labels = tf.matmul(y_true, self.col_label_vec)
        cat_label_mat = tf.tile(cat_labels, [1, self.num_classes])
        row_label_mat = tf.tile(self.row_label_vec, [batch_size, 1])
        if self.weightage == "linear":
            weight = tf.abs(cat_label_mat - row_label_mat)
        else:
            weight = (cat_label_mat - row_label_mat) ** 2
        numerator = tf.reduce_sum(weight * y_pred)
        label_dist = tf.reduce_sum(y_true, axis=0, keepdims=True)
        pred_dist = tf.reduce_sum(y_pred, axis=0, keepdims=True)
        w_pred_dist = tf.matmul(self.weight_mat, pred_dist, transpose_b=True)
        denominator = tf.reduce_sum(tf.matmul(label_dist, w_pred_dist))
        denominator /= tf.cast(batch_size, dtype=denominator.dtype)
        loss = tf.math.divide_no_nan(numerator, denominator)
        return tf.math.log(loss + self.epsilon)

    def get_config(self):
        config = {
            "num_classes": self.num_classes,
            "weightage": self.weightage,
            "epsilon": self.epsilon,
        }
        base_config = super().get_config()
        return {**base_config, **config}

In [4]:
#load the data
from model_trainer_and_tester import read_in_dataset_lstm, test_set_evaluate_multiple_lstm

shifts = [-2, -1, 0, 1, 2]

n_timesteps = len(shifts)
feature_count = len(features)

X_train, Y_train = read_in_dataset_lstm(features, shifts, to_read='train')
X_test, Y_test = read_in_dataset_lstm(features, shifts, to_read='test')
#train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

print(X_train.shape)
print(Y_train.shape)

# So, the model should be tanh activations and sigmoid outpiut layers...
# I'm going to guess both layers should be blstm... but that's it pretty much I think?

(47450, 5, 5)
(47450, 5, 1)


In [58]:
# I'm going to do sample weight here. For now I'll just set the weight to be... half of the number of weird samples?

sample_weight = np.ones(shape=(len(Y_train),))
# I'm gonna increase the weight by the inverse of the proportion of weird examples that there are
# How I define if there is a weird sample is by summing along the 2D squares to find where there's a 1, and then does a sum of times there's a 1
# I'm going to do n_timesteps times the inverse count frequency, because in the final version we only predict with the center value. So to correct for this I add this increase
new_weight = n_timesteps*len(Y_train)/np.sum(Y_train, axis=1).sum()

# Have to do a flatten() inside because of weird numpy stuff with a length 1 dimension
sample_weight[(np.sum(Y_train, axis=1) >= 1).flatten()] = new_weight

sample_weight

array([1., 1., 1., ..., 1., 1., 1.])

In [59]:
from tensorflow import keras

# TODO: Understand how the hell to build a keras model
model = Sequential()
# For the input number of units, I'll assume that number of timesteps * features is a good enough value
model.add(Bidirectional(LSTM(n_timesteps*feature_count, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
model.add(Bidirectional(LSTM(LSTM_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
# This last time distributed is super important, it follows the output structure of the paper I've been following closely
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='RMSprop',
              metrics=[keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')],
              weighted_metrics=[])

# train the model
history=model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=2,
                  #class_weight= {0:1, 1:10},
                  # sample_weight_mode='temporal',
                  sample_weight=sample_weight,
                  validation_split=0.1,
                  verbose=1
                  )

Epoch 1/2


Exception ignored in: <function UniquePtr.__del__ at 0x000001B51EBEBAF0>
Traceback (most recent call last):
  File "C:\Users\lolco\anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 74, in __del__
    self.deleter(obj)
KeyboardInterrupt: 


668/668 [==============================] - 22s 18ms/step - loss: 2.7194 - precision: 0.0134 - recall: 0.0134 - val_loss: 2.2672 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
668/668 [==============================] - 11s 16ms/step - loss: 2.5806 - precision: 0.0874 - recall: 0.0496 - val_loss: 2.2236 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [60]:
hist_results_dict = {}
acc_results_dict = {}
history.history

{'loss': [2.7193782329559326, 2.5805516242980957],
 'precision': [0.013399504125118256, 0.08741258829832077],
 'recall': [0.013379584066569805, 0.0495540127158165],
 'val_loss': [2.267162799835205, 2.2235677242279053],
 'val_precision': [0.0, 0.0],
 'val_recall': [0.0, 0.0]}

In [61]:
for num_layers in range(1, 5):
    # It's 31 per step, because it allows for there to be 16 trials, and for me to not go crazy
    for hidden_units in range(16, 513, 31):
        #print("Starting model: " + str((num_layers, hidden_units)))
        model = Sequential()
        # For the input number of units, I'll assume that number of timesteps * features is a good enough value
        for _ in range(num_layers):
            model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
        model.add(Bidirectional(LSTM(hidden_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
        # This last time distributed is super important, it follows the output structure of the paper I've been following closely
        model.add(TimeDistributed(Dense(1, activation='sigmoid')))
        model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                      metrics=[keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')],
                      weighted_metrics=[])

        # train the model
        history = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=20,
                  #class_weight= {0:1, 1:10},
                  # sample_weight_mode='temporal',
                  sample_weight=sample_weight,
                  validation_split=0.1,
                  verbose=0
                  )

        print("Finished model: " + str((num_layers, hidden_units)) + " with Precision: " + str(history.history['val_precision'][-1]))

        hist_results_dict[(num_layers, hidden_units)] = history
        # It's -1, to take the last accuracy measure obtained
        acc_results_dict[(num_layers, hidden_units)] = (history.history['val_precision'][-1], history.history['val_recall'][-1])

Finished model: (1, 16) with Precision: 0.0555555559694767
Finished model: (1, 47) with Precision: 0.054054055362939835
Finished model: (1, 78) with Precision: 0.0585365854203701
Finished model: (1, 109) with Precision: 0.06172839552164078
Finished model: (1, 140) with Precision: 0.05314009636640549
Finished model: (1, 171) with Precision: 0.06542056053876877
Finished model: (1, 202) with Precision: 0.0833333358168602
Finished model: (1, 233) with Precision: 0.0517241396009922
Finished model: (1, 264) with Precision: 0.05622490122914314
Finished model: (1, 295) with Precision: 0.05820105969905853
Finished model: (1, 326) with Precision: 0.03097345121204853
Finished model: (1, 357) with Precision: 0.04663212597370148
Finished model: (1, 388) with Precision: 0.03100775182247162
Finished model: (1, 419) with Precision: 0.03846153989434242
Finished model: (1, 450) with Precision: 0.10810811072587967
Finished model: (1, 481) with Precision: 0.03151862323284149
Finished model: (1, 512) with 

KeyboardInterrupt: 

In [62]:
acc_results_dict

{(1, 16): (0.0555555559694767, 0.0364583320915699),
 (1, 47): (0.054054055362939835, 0.03125),
 (1, 78): (0.0585365854203701, 0.0625),
 (1, 109): (0.06172839552164078, 0.02604166604578495),
 (1, 140): (0.05314009636640549, 0.0572916679084301),
 (1, 171): (0.06542056053876877, 0.0364583320915699),
 (1, 202): (0.0833333358168602, 0.02604166604578495),
 (1, 233): (0.0517241396009922, 0.03125),
 (1, 264): (0.05622490122914314, 0.0729166641831398),
 (1, 295): (0.05820105969905853, 0.0572916679084301),
 (1, 326): (0.03097345121204853, 0.0364583320915699),
 (1, 357): (0.04663212597370148, 0.046875),
 (1, 388): (0.03100775182247162, 0.02083333395421505),
 (1, 419): (0.03846153989434242, 0.0677083358168602),
 (1, 450): (0.10810811072587967, 0.02083333395421505),
 (1, 481): (0.03151862323284149, 0.0572916679084301),
 (1, 512): (0.03186274692416191, 0.0677083358168602),
 (2, 16): (0.0845070406794548, 0.03125),
 (2, 47): (0.07042253762483597, 0.02604166604578495),
 (2, 78): (0.03305784985423088, 0

In [74]:
pd.DataFrame(acc_results_dict, columns=acc_results_dict.keys(), index=['Precision', 'Recall']).transpose().to_csv('results_blstm.csv')

In [18]:
temp_y = model.predict(X_train)
temp_y

1483/1483 [==============================] - 5s 3ms/step


array([[[0.00627282],
        [0.01584293],
        [0.01336911],
        [0.02408313],
        [0.01425999]],

       [[0.00942823],
        [0.00914664],
        [0.01536419],
        [0.0124401 ],
        [0.00516671]],

       [[0.00379193],
        [0.00422932],
        [0.00482995],
        [0.00295994],
        [0.00844802]],

       ...,

       [[0.01065987],
        [0.02366466],
        [0.00473628],
        [0.0050675 ],
        [0.00676939]],

       [[0.02293292],
        [0.00628696],
        [0.00566995],
        [0.00733351],
        [0.00526068]],

       [[0.00610961],
        [0.00701646],
        [0.0087015 ],
        [0.00562242],
        [0.00662829]]], dtype=float32)

It's working which is good. It is giving garbage results though.